In [ ]:
import pandas as pd
import numpy as np
import time
import os
import tabula
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import xlsxwriter

In [ ]:
def timesleep(secs):
    if secs == 'rand200':
        secs = np.random.randint(100,200)
    elif secs == 'rand20':
        secs = np.random.randint(5,20)
    for t in range(0,secs):
        time.sleep(1)
        print(t+1, end=' -> ')
    print('\n')
        
def save_data(browser, suburb, property_data):
    results = browser.find_elements_by_xpath('//*[@id]')
    for ids in results:
        if ids.get_attribute('id').startswith('r'):
            address = browser.find_element_by_xpath('//*[@id="'+ids.get_attribute('id')+'"]/tbody/tr/td[2]/table/tbody/tr[1]/td/span[2]').text
            table_data = browser.find_element_by_xpath('//*[@id="'+ids.get_attribute('id')+'"]/tbody/tr/td[2]/table/tbody/tr[2]/td/table').text
            table_data = table_data.split('\n')
            property_data[address+suburb] = table_data
#             print('\n'+ids.get_attribute('id')+' ----> '+address+' ----> '+table_data[0])
    return browser, property_data

In [ ]:
def skip_pages(next_button, page):
    for row in next_button.find_elements_by_css_selector('td'):
        if row.text == str(page):
            next_page = row
            
    return next_page

In [ ]:
def check_skip_pages(browser, page):
    
    print('====== Skipping Pages =====')
    timesleep('rand20')
    Run = True
    current_page = 1
    while Run:
        next_button = browser.find_element_by_xpath('/html/body/center/table/tbody')
        if page > 10 and current_page == 1:
            print('====== Skipping to Page 10 =====')
            current_page = 10
            row = skip_pages(next_button, 10)
            row.click()
            timesleep('rand20')
            if 15 > page > 10:
                Run = False
        elif page > 15 and current_page == 10:
            current_page = 15
            row = skip_pages(next_button, 15)
            row.click()
            timesleep('rand20')
            if 20 > page > 15:
                Run = False
            print('====== Skipping to Page 15 =====')
        elif page > 20 and current_page == 15:
            current_page = 20
            row = skip_pages(next_button, 20)
            row.click()
            timesleep('rand20')
            if 25 > page > 20:
                Run = False
            print('====== Skipping to Page 20')
        elif page > 25 and current_page == 20:
            row = skip_pages(next_button, 25)
            row.click()
            timesleep('rand20')
            if 30 > page > 25:
                Run = False
            print('====== Skipping to Page 25')
    return browser


In [ ]:
def GET_SOLD_DATA(browser, url, suburb, property_data, page):
    #Start Searching sold properties
 
    Run = True
    browser = check_skip_pages(browser, page)
    try:
        while Run:
            browser, property_data = save_data(browser, suburb, property_data)
            timesleep('rand20')
            page +=1 #NextPage
            next_button = browser.find_element_by_xpath('/html/body/center/table/tbody')
            for row in next_button.find_elements_by_css_selector('td'):
                if row.text == str(page):
                    next_page = row
                    Run = True
                    break
                else:
                    Run = False
                
            print('========================================================= Page Number ---> '+str(page))
            next_page.click()
            timesleep('rand20')
    except Exception as e:
        print(e)
        print('its failing here')
        print('======================> '+suburb + ' ended loop @page # '+str(page))
        return property_data, page
    print('======================> '+suburb + ' ended loop @page # '+str(page))
    return property_data, page
    

    

In [ ]:
def checkupdate(suburb):
    print(suburb+' Up To Date')
    return

In [ ]:
# def main():
property_data = {}
postcodes = pd.read_csv('postcode.csv', index_col = 0)
url = 'http://house.ksou.cn/index.php'
log = []
for suburb, page in postcodes.values.tolist():
    if page >= 31:
        checkupdate(suburb) 
    else:
        try:
            browser = webdriver.Firefox()
            print('loading webpage...')
            browser.get(url)
            timesleep(5)

            inputElement = browser.find_element_by_id('q')
            button = browser.find_element_by_xpath('/html/body/center/form/table/tbody/tr/td[2]/input')
            inputElement.send_keys(suburb)
            print('searching for the '+suburb+'...')
            button.click()
            timesleep(5)
            main_window = browser.current_window_handle

            property_data, log = GET_SOLD_DATA(browser, url, suburb, property_data, page)
            browser.quit()

            property_df = pd.DataFrame.from_dict(property_data, orient='index')
            property_df.to_csv('property_data.csv')
            postcodes.loc[(postcodes.suburb == suburb),'page'] = log
            postcodes.to_csv('postcode.csv')
            timesleep('rand200')
        except Exception as e:
            print(e)
            print(suburb+' failed at '+str(page))

In [11]:
# if __name__ == "__main__":
#     main()

In [16]:
next_page = browser.find_element_by_link_text('6')
next_page.click()